<a href="https://colab.research.google.com/github/ShivamThapa243/Information-Retrieval/blob/main/positional_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Positional Inverted Index

In [1]:
# importing dataset from drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


*Check Point: Checking if the files are readable*

In [2]:
# accessing the dattaset
path = "/content/drive/MyDrive/Dataset/text_files/file1.txt"
with open(path, 'r') as file:
  content = file.read()

print(content)

Loving these vintage springs on my vintage strat. They have a good tension and great stability. If you are floating your bridge and want the most out of your springs than these are the way to go.


**Function definitions (include preprocessing functions)**

In [3]:
from types import new_class
def preprocessing(document):
  # 1. converting the string into lowercase and removing meta-tags
  content = document.lower()
  content = metadata_remover(content)

  # 2. converting string into tokens
  token_array = tokenizer_function(content)

  # 3. removing stopwords
  token_array = stopwords_remover_funtion(token_array)

  # 4. Punctuation remover function
  token_array = punctuation_remover_function(token_array)

  # 5. converting token array to string,
  # so that it can be stored as a string in the text file
  preprocessed_tokens = stringfy_token_array(token_array)

  # 6. removing extra spaces
  preprocessed_tokens = ' '.join(preprocessed_tokens.split())

  return preprocessed_tokens
# end of functtion


# function to convert the string into lower case and no meta data
def metadata_remover(content):
  tag = False
  quote = False
  str = ""
  for ch in content:
    if ch == '<' and not quote:
      tag = True
    elif ch == '>' and not quote:
      tag = False
    elif (ch == '"' or ch == "'") and tag:
      quote = not quote
    elif not tag:
      str = str + ch

  return str
# end of function

# function to convert string into tokens
def tokenizer_function(content):
  token_array = content.split()
  return token_array
# end of function

# funtion to remove stopwords from the token array
def stopwords_remover_funtion(token_array):
  stopwords = {'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll",
                 "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's",
                 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs',
                 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is',
                 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did',
                 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at',
                 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',
                 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again',
                 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both',
                 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same',
                 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've",
                 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn',
                 "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't",
                 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn',
                 "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"}
  new_array = []
  for token in token_array:
    if token in stopwords:
      continue
    else:
      new_array.append(token)
  return new_array
# end of funtion

# funtion to remove punctuation
def punctuation_remover_function(token_array):
  new_token = []
  puntuation = {'.', '?', '!', ',', ';', ':', '\'', '\"', '-', '_', ')', '(', '[', ']', '{', '}', '<', '>', '=', '/', '\\', '*', '#' }
  for token in token_array:
    temp = ""
    for ch in token:
      if  ch in puntuation:
        temp += " "
      else:
        temp += ch
    new_token.append(temp)
  return new_token
#end of function

# function to convert token array into string
def stringfy_token_array(token_array):
  token_string = ""
  token_string = ' '.join(token_array)
  return token_string
# end of function

*Check Point: Checking if the preprocessing stage is working fine*

In [4]:
val = "Loving these vintage springs on my vintage strat. They have a good tension and great stability. If you are floating your bridge and want the most out of your springs than these are the way to go."
print(val)
print(preprocessing(val))

Loving these vintage springs on my vintage strat. They have a good tension and great stability. If you are floating your bridge and want the most out of your springs than these are the way to go.
loving vintage springs vintage strat good tension great stability floating bridge want springs way go


**Preprocessing the file content and storing in a new directory**

In [5]:
import os

directory_path = "/content/drive/MyDrive/Dataset/text_files"
preprocessed_directory = "/content/drive/MyDrive/Preprocessed-Dataset"
# Create the preprocessed directory if it doesn't exist
os.makedirs(preprocessed_directory, exist_ok= True)

# List all files in the directory, excluding subdirectories
file_list = os.listdir(directory_path)

print(f"Files to process: {file_list}")

# iterating through the dataset and preprocessing each file present in the directory
for filename in file_list:
  file_path = os.path.join(directory_path, filename)
  # reading the original content of the dataset
  with open(file_path, 'r') as file:
    content = file.read()

  # preprocessing the original document
  preprocessed_content = preprocessing(content)

  # writing/ storing the preprocessed_content into the newly created directory
  preprocessed_file_path = os.path.join(preprocessed_directory, filename)
  with open(preprocessed_file_path, 'w') as preprocessed_file:
    preprocessed_file.write(preprocessed_content)
#end of for loop

Files to process: ['file223.txt', 'file761.txt', 'file14.txt', 'file265.txt', 'file949.txt', 'file365.txt', 'file970.txt', 'file740.txt', 'file249.txt', 'file982.txt', 'file554.txt', 'file728.txt', 'file568.txt', 'file797.txt', 'file938.txt', 'file41.txt', 'file956.txt', 'file891.txt', 'file503.txt', 'file218.txt', 'file237.txt', 'file316.txt', 'file224.txt', 'file674.txt', 'file92.txt', 'file603.txt', 'file817.txt', 'file109.txt', 'file709.txt', 'file304.txt', 'file784.txt', 'file405.txt', 'file487.txt', 'file841.txt', 'file736.txt', 'file924.txt', 'file807.txt', 'file52.txt', 'file847.txt', 'file913.txt', 'file389.txt', 'file813.txt', 'file253.txt', 'file352.txt', 'file471.txt', 'file921.txt', 'file25.txt', 'file142.txt', 'file587.txt', 'file570.txt', 'file412.txt', 'file888.txt', 'file542.txt', 'file991.txt', 'file126.txt', 'file735.txt', 'file119.txt', 'file343.txt', 'file945.txt', 'file574.txt', 'file756.txt', 'file540.txt', 'file174.txt', 'file942.txt', 'file967.txt', 'file638.tx

Printing 5 Original documents before and after preprocessing

In [7]:
preprocessed_directory_path = "/content/drive/MyDrive/Preprocessed-Dataset"
preprocessed_file_list = os.listdir(preprocessed_directory_path)

file_count = 1

for filename in preprocessed_file_list:
  if file_count > 5:
    break

  preprocessed_file_path = os.path.join(preprocessed_directory_path, filename)
  with open(preprocessed_file_path, 'r') as file:
    preprocessed_content = file.read()

  print(file_count, f" {filename}")
  print(preprocessed_content, "\n")
  file_count += 1

1  file732.txt
tha best leads 

2  file214.txt
lot reviews divided oh sounds great sounds like s ty gong come mind bought anyways definitely cheap buy still looks like amazing cymbal sound sound pretty crappy still great buy beginner drummers drummer like jam basements garage start get gig two consider going better cymbal update i ve cymbal 1 5 years cracked point can t play anymore likely way i ve hitting it crappy manufacturing look old review think bit critical i ve grown love bright sound cymbal probably buy another one 

3  file605.txt
application plug three guitars pedalboard see pics noiseless shows tone sucking qualities all runs well 9 volt dunlop dc brick although function lights also works power but lights come handy know selection on 

4  file90.txt
solid strap used strat nice flashy fits style well works well solidly made worries guitar going come playing i ve taken shots poly locks check mechanism pretty nice definitely want last good long while 

5  file186.txt
bought tw